In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath("."))

with open("keys/openai_api_key.txt") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

In [2]:
import argparse
from datetime import datetime

import numpy as np
import wandb
from src.generate import generate_prompt
from src.llms import GPT, Model
from xarc import TASK_LIST
from xarc import generate_data_with_options as xarc_generate

from utils import cross_entropy_loss